In [9]:
import sys
import os 
import re
import lib.database_module as db
import lib.download_mine as down
import lib.encoding_module as en
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def Search(search_list, db_location):    # print command line arguments
    pickled_transformer = joblib.load('data/vectorizer.pkl')
    search_vector = en.get_searchterm_vector(pickled_transformer,search_list[0])
    pages_vectors = db.select_all_page_vectors(location=db_location)
    vectors = [x[1] for x in pages_vectors]
    page_ids = [x[0] for x in pages_vectors]
    

In [13]:
search_list = "Bayes"
db_location = 'local'
pickled_transformer = joblib.load('data/vectorizer.pkl')
search_vector = en.get_searchterm_vector(pickled_transformer,search_list[0])
pages_vectors = db.select_all_page_vectors(location=db_location)
vectors = [x[1] for x in pages_vectors]
page_ids = [x[0] for x in pages_vectors]

In [17]:
pages_vectors[3]

(676555,
 [0.0011746330803574,
  0.0061037577313836,
  -0.000157804087470463,
  0.00196012269692119,
  0.00941475015153055,
  -0.013012812078798,
  -0.000542260828221871,
  -0.0081548233702708,
  -0.00103290968500331,
  -0.000678648074152639,
  -0.00315987877963146,
  0.000592005767047315,
  2.61714620680391e-05,
  0.0019903751490587,
  2.94343511944512e-05])

In [10]:
cos = cosine_similarity(vectors,search_vector[search_vector.keys()[0]])
indices = cos.argsort(axis = 0)[-5:].ravel().tolist()
indices.reverse()
page_match_list = [(page_ids[i], cos[i]) for i in indices]
temp_list =[]

connection, cur, _ = db.connect_to_postgres(db_location)
for page,score in page_match_list:
    sql = """SELECT title, page
             FROM page
             WHERE page_id = {};""".format(page)
    cur.execute(sql)
    temp = cur.fetchone()
    connection.commit()
    #print "Page title: {}\n{}\n\n".format(temp[0], temp[1][:250])
    temp_list.append( (temp[0], temp[1][:280], score))

cur.close()
connection.close()

for result in temp_list:
    print "Page title: {}  Match Score: {:.2f}\n{}\n\n".format(result[0],(result[2][0])*100,result[1][:280].replace("\n"," "))